<a href="https://colab.research.google.com/github/CarlosfcPinheiro/pibic-api-llm-integration/blob/main/pibic_teste_aplicacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Definição dos prompts e endpoint =======================
# Essa seção aqui pode ser feita em outro colab
import requests
import time
import csv

api_url = "https://zetta-faunlike-kindheartedly.ngrok-free.dev"
ENDPOINT_TEST = f"{api_url}/summarize"

prompts = [
    """A energia solar tem se destacado como uma das fontes renováveis mais promissoras para o futuro energético do planeta. Com a crescente preocupação em relação às mudanças climáticas e à necessidade de reduzir a dependência de combustíveis fósseis, muitos países têm investido em tecnologias solares para gerar eletricidade de forma limpa e sustentável. O Brasil, por exemplo, possui um dos maiores potenciais solares do mundo, especialmente nas regiões Nordeste e Centro-Oeste, onde a incidência de radiação solar é alta durante todo o ano. Nos últimos anos, houve um aumento significativo na instalação de painéis solares em residências, comércios e indústrias, impulsionado por incentivos fiscais e pela redução dos custos dos equipamentos. Além disso, a geração distribuída tem permitido que consumidores se tornem também produtores de energia, contribuindo para a descentralização do sistema elétrico nacional. Apesar dos avanços, ainda existem desafios a serem superados, como a necessidade de investimentos em infraestrutura, armazenamento de energia e políticas públicas mais robustas. No entanto, com o avanço da tecnologia e o engajamento da sociedade, a energia solar tende a ocupar um papel cada vez mais relevante na matriz energética brasileira, promovendo desenvolvimento sustentável e redução de impactos ambientais.""",

    """A cidade de Ouro Preto, localizada no estado de Minas Gerais, é um dos maiores símbolos da arquitetura colonial brasileira e da história do país. Fundada no século XVII durante o ciclo do ouro, ela rapidamente se tornou um dos principais centros econômicos da colônia portuguesa, atraindo milhares de pessoas em busca de riqueza. Suas ruas de pedra, ladeiras íngremes e construções preservadas revelam um passado marcado por opulência, religiosidade e resistência. Ouro Preto abriga algumas das mais impressionantes igrejas barrocas do Brasil, como a Igreja de São Francisco de Assis, projetada por Aleijadinho, e decorada com obras de Mestre Ataíde. O município também foi palco de importantes eventos da Inconfidência Mineira, movimento que buscava a independência do Brasil em relação a Portugal. Hoje, a cidade é reconhecida como Patrimônio Mundial pela UNESCO, sendo um destino turístico que atrai visitantes interessados em história, arte e cultura. Além de seu valor histórico, Ouro Preto possui uma vibrante vida universitária, graças à Universidade Federal de Ouro Preto (UFOP), e promove diversos festivais de arte, música e cinema ao longo do ano. Essa combinação de tradição e juventude faz da cidade um espaço único, onde o passado e o presente convivem em harmonia.""",

    """A inteligência artificial (IA) tem se consolidado como uma das tecnologias mais transformadoras do século XXI, impactando profundamente diversos setores da sociedade. Desde a medicina, onde algoritmos auxiliam no diagnóstico precoce de doenças, até a educação, com sistemas personalizados de ensino adaptativo, a IA está redefinindo a forma como interagimos com o mundo. Na indústria, robôs inteligentes otimizam linhas de produção, enquanto no comércio, assistentes virtuais melhoram a experiência do consumidor. Com algoritmos cada vez mais sofisticados e modelos de linguagem avançados, como os LLMs, a IA também está presente em áreas criativas, gerando textos, imagens e até composições musicais. No entanto, essa evolução tecnológica levanta questões éticas importantes. O uso responsável dos dados, a transparência nos processos de decisão automatizada e o combate ao viés algorítmico são temas centrais em debates acadêmicos e políticos. Além disso, há preocupações sobre o impacto da IA no mercado de trabalho, com a automação substituindo funções humanas em ritmo acelerado. Para garantir que os benefícios da inteligência artificial sejam amplamente distribuídos, é essencial que governos, empresas e sociedade civil colaborem na criação de políticas públicas robustas, promovendo inclusão digital, educação tecnológica e regulamentações que assegurem o uso ético e seguro da IA."""
]

In [21]:
# Teste de tempo de resposta ===============
import statistics
import numpy as np
# cada prompt é testado e armazenado em um .csv contendo as colunas de: prompt de entrada, resposta de saída, tamanho do prompt e tempo de execução

# Para esse modelo a RESPOSTA DE SAÍDA é indiferente, pois possui parâmetros padrões determinísticos, evitando variações criativas
# cada prompt deve fazer 30 requisições para que seja calculada e analisadas métricas como:
# - média de tempo de prompt => tempo médio por requisição
# - valores máximos e mínimos => valores máximos e mínimos de tempos de requisição para cada prompt
# - desvio padrão => informa a dispersão absoluta dos dados em relação à média
# - coeficiente de variação => informa o quão dispersos estão os dados relativos à média
# - percentis ???

# Tem problema os prompts resultarem na mesma saída, dado uma mesma entrada? (justificativa do modelo). Seria importante que o modelopossua variações criativas?

csv_filename = "tempo_requisicoes.csv"

REQ_PER_PROMPT = 30
# dicionário com o array de todos os tempos de requisição
TIMES_REQ_PER_PROMPT_ALL = {}

# Adicionar try catch aqui para resposta com exceção
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
  writer = csv.writer(file)
  writer.writerow([
      "prompt/entrada",
      # "saída/resposta",
      "tamanho_prompt",
      "media_tempo_s",
      "tempo_total_s",
      "req_por_prompt",
      "tempo_req_min_s",
      "tempo_req_max_s",
      "desvio_padrao_s",
      "coef_var_%"
  ])

  for idx, prompt in enumerate(prompts, start=1):
    times_req_per_prompt = []
    for i in range(REQ_PER_PROMPT):
      try:
        start = time.time()
        response = requests.post(ENDPOINT_TEST, json={"text":prompt})
        end = time.time()

        text_response = response.json()
        total_time = round(end-start, 3)

        times_req_per_prompt.append(total_time)

      except Exception as e:
          print("Houve um problema com o teste das requisições", e)
          break

    # guardar os tempos de requisição no dicionário
    TIMES_REQ_PER_PROMPT_ALL[idx] = times_req_per_prompt

    reqs_total_time = round(sum(times_req_per_prompt), 3)
    avg_total_time = round(statistics.mean(times_req_per_prompt), 4)
    min_time = min(times_req_per_prompt)
    max_time = max(times_req_per_prompt)
    deviation = round(statistics.pstdev(times_req_per_prompt),  4)
    coef_var = round(deviation/avg_total_time, 4)

    # escrever csv com as métricas para as requisições de cada prompt
    writer.writerow([
      prompt,
      # text_response["summary"],
      len(prompt),
      avg_total_time,
      reqs_total_time,
      REQ_PER_PROMPT,
      min_time,
      max_time,
      deviation,
      coef_var
    ])
    print(f"PROMPT {idx}")
    print(f"Tempo total de {REQ_PER_PROMPT} prompts: {reqs_total_time}")
    print(f"Tempo médio de prompt: {avg_total_time}\n")

PROMPT 1
Tempo total de 30 prompts: 103.154
Tempo médio de prompt: 3.4385

PROMPT 2
Tempo total de 30 prompts: 140.124
Tempo médio de prompt: 4.6708

PROMPT 3
Tempo total de 30 prompts: 140.609
Tempo médio de prompt: 4.687



In [22]:
from google.colab import data_table
import pandas as pd

data_table.enable_dataframe_formatter()
df = pd.read_csv("tempo_requisicoes.csv")

print("\nDetalhes de cada requisição:")
display(df)


Detalhes de cada requisição:


,prompt/entrada,tamanho_prompt,media_tempo_s,tempo_total_s,req_por_prompt,tempo_req_min_s,tempo_req_max_s,desvio_padrao_s,coef_var_%
0,A energia solar tem se destacado como uma das ...,1329,3.4385,103.154,30,2.572,7.530,1.0202,0.2967
1,"A cidade de Ouro Preto, localizada no estado d...",1279,4.6708,140.124,30,3.420,7.743,1.1816,0.2530
2,A inteligência artificial (IA) tem se consolid...,1390,4.6870,140.609,30,2.552,23.218,3.9677,0.8465
